# Players Pool Handler

In [6]:
import time

from itertools import chain

import requests as req
import pandas as pd
import numpy as np

from settings import TOKEN

## 1. Master leagues

In [7]:
def get_players_master():
    container = []
    leagues = [
        "challengerleagues",
        "grandmasterleagues",
        "masterleagues"
    ]
    base_url = "https://euw1.api.riotgames.com/lol/league/v4"
    for league in leagues:  
        url = f"{base_url}/{league}/by-queue/RANKED_SOLO_5x5?api_key={TOKEN}"
        res = req.get(url)
        if res.status_code == 200:
            container.append(res.json())
        else:
            print(f"Something went wrong {res.status_code}")
            break
    return container

- Execute the function

In [3]:
players_master = get_players_master()

- We get an array of the top 3 leagues
- Each league has its own number of entries which are the players
- In the players object there is no "tier" attribute, we need to attacht it

In [4]:
print(players_master[0].keys())
players_master[0]["entries"][0]

dict_keys(['tier', 'leagueId', 'queue', 'name', 'entries'])


{'summonerId': 'QTnCkk_9K6l-FASK1MUNwQ-VSFi06VS96T91Sc95Hl_UfJc',
 'summonerName': 'SK LlMIT',
 'leaguePoints': 1066,
 'rank': 'I',
 'wins': 436,
 'losses': 375,
 'veteran': True,
 'inactive': False,
 'freshBlood': False,
 'hotStreak': False}

- Attaching the tier to each entry

In [5]:
for x in players_master:
    for y in x["entries"]:
        y["tier"] = x["tier"]

In [6]:
players_master[0]["entries"][0]

{'summonerId': 'QTnCkk_9K6l-FASK1MUNwQ-VSFi06VS96T91Sc95Hl_UfJc',
 'summonerName': 'SK LlMIT',
 'leaguePoints': 1066,
 'rank': 'I',
 'wins': 436,
 'losses': 375,
 'veteran': True,
 'inactive': False,
 'freshBlood': False,
 'hotStreak': False,
 'tier': 'CHALLENGER'}

- Now we want to flatten all leagues into one list because we already specified the league on each entry

In [7]:
players_master = list(chain.from_iterable([x["entries"] for x in players_master]))

In [8]:
len(players_master)

2945

## 2. Student leagues (This time only Diamond + )

- Each page gives us around 200 players
- We should loop on a range and break the loop once the number of players retrieved is 0

In [31]:
def get_players_student(queue: str, tiers: dict):
    base_url = "https://euw1.api.riotgames.com/lol/league/v4/entries"
    student_players = []
    n_requests = 0
    for tier in tiers:
        for division in tiers[tier]:
            pages = 0
            while True:
                # control the number of requests
                if n_requests >= 100:
                    print(f"Number of Entries so far: {len(student_players)}")
                    n_requests = 0
                    time.sleep(121)
                pages += 1
                url = f"{base_url}/{queue}/{tier}/{division}?page={pages}&api_key={TOKEN}"
                res = req.get(url)
                n_requests += 1
                if res.status_code == 200:
                    if len(res.json()) > 0:
                        student_players.append(res.json())
                    else:
                        break
                else:
                    print(f"Something went wrong {res.status_code}")
                    break
                    
    return student_players

- Execute the function

In [34]:
# first one is a lot of entries.
#players_student = get_players_student("RANKED_SOLO_5x5", {"DIAMOND": ["I", "II"], "PLATINUM": "I"})
players_student = get_players_student("RANKED_SOLO_5x5", {"DIAMOND": "I"})

Number of Entries so far: 98
Number of Entries so far: 198


In [36]:
len(players_student)

250

- Now we want to flatten all the pages into one big list.

In [37]:
chained = list(chain.from_iterable(players_student))

In [40]:
chained[0]

{'leagueId': 'ade103fb-f7fb-4ca5-bbe1-c3a95b4a219f',
 'queueType': 'RANKED_SOLO_5x5',
 'tier': 'DIAMOND',
 'rank': 'I',
 'summonerId': 'rvTHik0BLi0tKf0oBNH1LB-MrXNHgTYTIJp_xM9Jl22xfFpp',
 'summonerName': 'Meiran',
 'leaguePoints': 90,
 'wins': 200,
 'losses': 187,
 'veteran': False,
 'inactive': False,
 'freshBlood': False,
 'hotStreak': True}

In [41]:
len(chained)

50927

## 3. Give it a clean format and merge both lists

### The important data points:

- summonerId
- summonerName
- leaguePoints
- rank
- wins
- losses
- tier

- Drop uncommon columns

In [15]:
players_df = pd.DataFrame(players_student).drop(columns=["leagueId", "miniSeries", "queueType"])

In [16]:
players_df.shape

(5600, 11)

- Concatenate the data into one big player list

In [17]:
players_df = pd.concat([players_df, pd.DataFrame(players_master)])

## 4. Check the data

In [18]:
print(f"Columns: {players_df.shape[1]} Rows: {players_df.shape[0]}\n")
print(f"Number of duplcated rows: {players_df.duplicated().sum()}\n")
print("Null values per column:\n")
print(players_df.isna().sum())
print("Data types:\n")
print(players_df.dtypes)

Columns: 11 Rows: 8545

Number of duplcated rows: 135

Null values per column:

tier            0
rank            0
summonerId      0
summonerName    0
leaguePoints    0
wins            0
losses          0
veteran         0
inactive        0
freshBlood      0
hotStreak       0
dtype: int64
Data types:

tier            object
rank            object
summonerId      object
summonerName    object
leaguePoints     int64
wins             int64
losses           int64
veteran           bool
inactive          bool
freshBlood        bool
hotStreak         bool
dtype: object


- Remove duplicated rows

In [19]:
players_df.loc[players_df["summonerId"] == "vL_isz55-r376XwG2YK8avzmHmYN815ZuwxvohHxXM3PeVTS"]

,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
200,DIAMOND,I,vL_isz55-r376XwG2YK8avzmHmYN815ZuwxvohHxXM3PeVTS,Shacpai,0,368,347,False,False,False,False
205,DIAMOND,I,vL_isz55-r376XwG2YK8avzmHmYN815ZuwxvohHxXM3PeVTS,Shacpai,0,368,347,False,False,False,False


In [20]:
players_df = players_df.drop_duplicates()

In [21]:
print(f"Columns: {players_df.shape[1]} Rows: {players_df.shape[0]}\n")
print(f"Number of duplcated rows: {players_df.duplicated().sum()}\n")
print("Null values per column:\n")
print(players_df.isna().sum())

Columns: 11 Rows: 8410

Number of duplcated rows: 0

Null values per column:

tier            0
rank            0
summonerId      0
summonerName    0
leaguePoints    0
wins            0
losses          0
veteran         0
inactive        0
freshBlood      0
hotStreak       0
dtype: int64


## 5. Drop useless columns

- veteran
- inactive
- freshBlood
- hotStreak

In [22]:
players_df = players_df.drop(columns=["veteran", "inactive", "freshBlood", "hotStreak"])

In [23]:
players_df.head()

,tier,rank,summonerId,summonerName,leaguePoints,wins,losses
0,DIAMOND,I,rvTHik0BLi0tKf0oBNH1LB-MrXNHgTYTIJp_xM9Jl22xfFpp,Meiran,90,200,187
1,DIAMOND,I,Q1MxBiPaWxpvLUOcHTyUTGIRLoOsQ0pPW2GXJpFzlShKg4J_,Preciöus,0,101,79
2,DIAMOND,I,UHd2YrByzhM458uxwF_sR5saxARizi-quOa-S30OmFH41Or8,ˆˇˇˆˇˇˆ,52,160,146
3,DIAMOND,I,JbkMW-xXj99fD7CesxP-4Q7HTbTVw_DjEAXhMuPMGqj1xEZt,Huskigodx,3,264,261
4,DIAMOND,I,2RiE8ql63iwtSSYQYA_f84PtrCX_js2FrAWlOJfI4Wm83uES,Ghiosu,30,448,434


## 6. Export the data

In [25]:
players_df.to_pickle("raw_data/players_pool.pkl", protocol=4)